In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

## Constructs and runs Grover's algorithm for a 2-qubit system targeting a specific state ('00', '01', '10', or '11').

In [2]:
def run_grover(target_state):
    # Initialize 2 qubits and 2 classical bits
    qc = QuantumCircuit(2, 2)

    # 1. State Preparation: Apply Hadamard to put qubits in superposition
    qc.h([0, 1])
    qc.barrier()

    # 2. Oracle: Marks the target state by flipping its phase
    # (Implementation of specific Phase Oracles for 2 qubits)
    if target_state == '11':
        qc.cz(0, 1) # Flips phase if 11
    elif target_state == '10':
        qc.z(1)     # Flips q1 to 1 temporarily if needed, here just Z on 1 acts if 10 if preceded/followed properly
        # Standard CZ approach for 10: Wrap q0 in X gates? No, q1 is MSB usually.
        # Let's use a standard construct: X gates to match '11' condition
        qc.x(0)
        qc.cz(0, 1)
        qc.x(0)
    elif target_state == '01':
        qc.x(1)
        qc.cz(0, 1)
        qc.x(1)
    elif target_state == '00':
        qc.x([0, 1])
        qc.cz(0, 1)
        qc.x([0, 1])
    
    qc.barrier()

    # 3. Diffuser (Reflection about the mean)
    # Apply H, X, Z, X, H sequence generally
    qc.h([0, 1])
    qc.x([0, 1])
    qc.cz(0, 1)
    qc.x([0, 1])
    qc.h([0, 1])
    qc.barrier()

    # 4. Measure
    qc.measure([0, 1], [0, 1])

    return qc

In [3]:
# --- Execution ---
# You can change this target to '00', '01', '10', or '11'
target = '00' 
qc = run_grover(target)

print(f"--- Circuit for Target {target} ---")
print(qc.draw())

--- Circuit for Target 00 ---
     ┌───┐ ░ ┌───┐   ┌───┐ ░ ┌───┐┌───┐   ┌───┐┌───┐ ░ ┌─┐   
q_0: ┤ H ├─░─┤ X ├─■─┤ X ├─░─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░─┤M├───
     ├───┤ ░ ├───┤ │ ├───┤ ░ ├───┤├───┤ │ ├───┤├───┤ ░ └╥┘┌─┐
q_1: ┤ H ├─░─┤ X ├─■─┤ X ├─░─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░──╫─┤M├
     └───┘ ░ └───┘   └───┘ ░ └───┘└───┘   └───┘└───┘ ░  ║ └╥┘
c: 2/═══════════════════════════════════════════════════╩══╩═
                                                        0  1 


In [4]:
# Simulate
simulator = AerSimulator()
# Execute sufficient shots to demonstrate probability (e.g., 1024)
result = simulator.run(qc, shots=1024).result()
counts = result.get_counts(qc)

print(f"\nResults for target {target}:", counts)
# Ideally, counts should be {'00': 1024} for perfect Grover on 2 qubits.

# Plot (This will generate a window/image in a local environment)
plot_histogram(counts)
plt.show()


Results for target 00: {'00': 1024}
